# Persisted incremental DataSlice basics

In [214]:
# @title Imports
import os
import pytz
import tempfile

from koladata import kd
from koladata import kd_ext

In [215]:
# @title Utilities, e.g. for printing diagnostics

DataSliceManagerView = kd_ext.persisted_data.DataSliceManagerView
PersistedIncrementalDataSliceManager = (
    kd_ext.persisted_data.PersistedIncrementalDataSliceManager
)


def get_bag_name_to_size_on_disk_in_mb_dict(
    manager: PersistedIncrementalDataSliceManager,
) -> dict[str, float]:
  result = {}
  data_bag_manager = manager._data_bag_manager
  for bag_metadata in data_bag_manager._metadata.data_bag_metadata:
    bag_name = bag_metadata.name
    stat_result = os.stat(data_bag_manager._get_bag_filepath(bag_name))
    result[bag_name] = stat_result.st_size / 1024 / 1024
  return result


def format_percent(numerator, denominator) -> str:
  return f'{numerator / denominator * 100:.2f}%'


def print_diagnostics(
    manager: PersistedIncrementalDataSliceManager, heading: str
):
  bag_manager = manager._data_bag_manager
  available_bag_names = bag_manager.get_available_bag_names()
  loaded_bag_names = bag_manager.get_loaded_bag_names()
  approximate_available_triples = sum([
      bag_manager._read_bag_from_file(bag_name).get_approx_size()
      for bag_name in available_bag_names
  ])
  approximate_loaded_triples = sum([
      bag_manager.get_minimal_bag({bag_name}).get_approx_size()
      for bag_name in loaded_bag_names
  ])
  bag_name_to_size_on_disk_in_mb = get_bag_name_to_size_on_disk_in_mb_dict(
      manager
  )
  loaded_bags_size_on_disk_mb = sum([
      bag_name_to_size_on_disk_in_mb[bag_name] for bag_name in loaded_bag_names
  ])
  available_bags_size_on_disk_mb = sum([
      bag_name_to_size_on_disk_in_mb[bag_name]
      for bag_name in bag_manager.get_available_bag_names()
  ])
  print(heading)
  print('=' * len(heading))
  print(
      f'Managed bags: {len(available_bag_names)}   triples:'
      f' {approximate_available_triples}   size on disk:'
      f' {available_bags_size_on_disk_mb:.2f} Mb'
  )
  print(
      'Loaded bags: '
      f' {len(loaded_bag_names)} ({format_percent(len(loaded_bag_names), len(available_bag_names))})'
      '   triples:'
      f' {approximate_loaded_triples} ({format_percent(approximate_loaded_triples, approximate_available_triples)})'
      f'  size on disk: {loaded_bags_size_on_disk_mb:.2f} Mb'
      f' ({format_percent(loaded_bags_size_on_disk_mb, available_bags_size_on_disk_mb)})'
  )
  print()

## Creating a persisted incremental DataSlice

Consider the following plain Koda DataSlice:

In [216]:
query_schema = kd.named_schema('query')
new_query = query_schema.new
doc_schema = kd.named_schema('doc')
new_doc = doc_schema.new

query_ds = kd.slice([
    new_query(
        id=1,
        text='How high is the Eiffel tower',
        doc=kd.list([
            new_doc(id=10, title='Attractions of Paris', content='foo' * 10000)
        ]),
    ),
    new_query(
        id=2,
        text='How high is the empire state building',
        doc=kd.list([
            new_doc(
                id=11, title='Attractions of New York', content='bar' * 10000
            ),
            new_doc(
                id=12,
                title="The world's tallest buildings",
                content='baz' * 10000,
            ),
        ]),
    ),
])

It is simple to turn it into a version that is persisted to disk and whose data can be loaded incrementally on demand.

Persistence and incremental loading is useful for situations where the full data does not fit into RAM. All the data is persisted to disk, and only the parts that are needed get loaded into RAM.

The examples in this notebook all have tiny data, but the techniques shown here also apply to real-world applications where the data is large.

In [217]:
manager = PersistedIncrementalDataSliceManager.create_new(tempfile.mkdtemp())

root = DataSliceManagerView(manager)
root.query = query_ds.implode(ndim=-1), 'Added initial "query" data'

The assignment to `root.query` received the full query data `query_ds.implode(...)` in
memory, alongside a commit message. The manager decomposed the query data into smaller DataBags, persisted the bags,
and kept them in a cache. We clear the cache to make the memory footprint of
the manager tiny.

In [218]:
print_diagnostics(manager, 'After adding initial "query" data')

After adding initial "query" data
Managed bags: 10   triples: 30   size on disk: 0.01 Mb
Loaded bags:  10 (100.00%)   triples: 30 (100.00%)  size on disk: 0.01 Mb (100.00%)



In [219]:
manager.clear_cache()
print_diagnostics(manager, 'After clearing the cache')

After clearing the cache
Managed bags: 10   triples: 30   size on disk: 0.01 Mb
Loaded bags:  1 (10.00%)   triples: 0 (0.00%)  size on disk: 0.00 Mb (2.38%)



## Getting data from a persisted incremental DataSlice

At this point no data triples are loaded, but the manager always knows the schema of the data.

In [220]:
manager.get_schema()

DataItem(ENTITY(
  query=LIST[query(doc=LIST[doc(content=STRING, id=INT32, title=STRING)], id=INT32, text=STRING)],
), schema: SCHEMA, bag_id: $0b59)

In [221]:
root.get_schema()

DataItem(ENTITY(
  query=LIST[query(doc=LIST[doc(content=STRING, id=INT32, title=STRING)], id=INT32, text=STRING)],
), schema: SCHEMA, bag_id: $0b59)

Asking for data will load it into memory.

In [222]:
root.query[:].doc[:].id.get_data_slice()

DataSlice([[10], [11, 12]], schema: INT32, present: 3/3, bag_id: $36e3)

In [223]:
print_diagnostics(manager, 'After getting the DataSlice with doc ids')

After getting the DataSlice with doc ids
Managed bags: 10   triples: 30   size on disk: 0.01 Mb
Loaded bags:  6 (60.00%)   triples: 16 (53.33%)  size on disk: 0.00 Mb (23.68%)



By default, it loads only stubs of the data.

In [224]:
root.query[:].doc[:].get_data_slice()

DataSlice([
  [Entity():$004EUVCZgrH8UPSeOmQyzE],
  [Entity():$004EUVCZgrH8UPSeOmQyzH, Entity():$004EUVCZgrH8UPSeOmQyzI],
], schema: doc(), present: 3/3, bag_id: $9167)

You can ask explicitly to populate the descendants.

In [225]:
root.query[:].doc[:].get_data_slice(
    populate_including_descendants=[root.query[:].doc[:]]
)

DataSlice([
  [
    Entity(
      content='foofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofo'...'oofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoo',
      id=10,
      title='Attractions of Paris',
    ),
  ],
  [
    Entity(
      content='barbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarba'...'arbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbar',
      id=11,
      title='Attractions of New York',
    ),
    Entity(
      content='bazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazba'...'azbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazba

In [226]:
print_diagnostics(manager, 'After getting all doc data')

After getting all doc data
Managed bags: 10   triples: 30   size on disk: 0.01 Mb
Loaded bags:  8 (80.00%)   triples: 24 (80.00%)  size on disk: 0.01 Mb (91.52%)



You can also ask for the data of the ancestors up to the root. What you get is a minimal skeleton of the ancestors. For example, in the next cell you do not get the query text.

In [227]:
root.get_data_slice(populate_including_descendants=[root.query[:].doc[:]])

DataItem(Entity(
  query=List[
    Entity(
      doc=List[
        Entity(
          content='foofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofo'...'oofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoo',
          id=10,
          title='Attractions of Paris',
        ),
      ],
    ),
    Entity(
      doc=List[
        Entity(
          content='barbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarba'...'arbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbarbar',
          id=11,
          title='Attractions of New York',
        ),
        Entity(
          content='bazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazbazba

In [228]:
print_diagnostics(manager, 'After getting all doc data and the minimal skeleton of doc ancestors up to the root')

After getting all doc data and the minimal skeleton of doc ancestors up to the root
Managed bags: 10   triples: 30   size on disk: 0.01 Mb
Loaded bags:  8 (80.00%)   triples: 24 (80.00%)  size on disk: 0.01 Mb (91.52%)



## Updating a persisted incremental DataSlice

It is simple to add or override data.

In [229]:
query = root.query[:]

query.locale = (
    kd.slice(['en-GB', 'en-US']),
    'Added locale information to queries',
)

query.text = (
    kd.slice(['How high is the Eiffel tower in Paris', None])
    | query.text.get_data_slice(),
    'Updated the text of the first query about the Eiffel tower',
)

In [230]:
query.locale.get_data_slice()

DataSlice(['en-GB', 'en-US'], schema: STRING, present: 2/2, bag_id: $ff7d)

In [231]:
query.text.get_data_slice()

DataSlice(['How high is the Eiffel tower in Paris', 'How high is the empire state building'], schema: STRING, present: 2/2, bag_id: $c1c9)

## Looking at the revision history

In [232]:
manager.get_revision_history(tz=pytz.timezone('Europe/Zurich'))

[CreationMetadata(description='Initial state with an empty root', timestamp='2026-02-03 14:06:36 CET'),
 AttributeUpdateMetadata(description='Added initial "query" data', timestamp='2026-02-03 14:06:37 CET', at_path='', attr_name='query'),
 AttributeUpdateMetadata(description='Added locale information to queries', timestamp='2026-02-03 14:06:40 CET', at_path='.query[:]', attr_name='locale'),
 AttributeUpdateMetadata(description='Updated the text of the first query about the Eiffel tower', timestamp='2026-02-03 14:06:41 CET', at_path='.query[:]', attr_name='text')]

## Branching

Any manager can be branched. Branches are useful for rolling back updates, for what-if experiments, and for filtered versions of the data.

### Branching to roll back updates

In [233]:
query.text.get_data_slice()

DataSlice(['How high is the Eiffel tower in Paris', 'How high is the empire state building'], schema: STRING, present: 2/2, bag_id: $aecf)

In [234]:
before_updating_query_text = manager.branch(
    tempfile.mkdtemp(),
    # Note that we pass revision_history_index=2. The revision with index 2 in
    # manager.get_revision_history(...) from above has a description that says:
    # "Added locale information to queries". We will create a branch on top of
    # that, i.e. the branch will contain the locale information, but it will not
    # include the effect of the revision with index 3 (whose description is
    # "Updated the text of the first query about the Eiffel tower").
    revision_history_index=2,
    description='Branch to roll back state',
)

In [235]:
root_before_updating_query_text = DataSliceManagerView(before_updating_query_text)

root_before_updating_query_text.query[:].text.get_data_slice()

DataSlice(['How high is the Eiffel tower', 'How high is the empire state building'], schema: STRING, present: 2/2, bag_id: $ae34)

Updates to the branch do not affect the original manager, and vice versa. The two managers evolve independently.

In [236]:
root_before_updating_query_text.query[:].text = (
    kd.slice(['How heavy is the Eiffel Tower', None])
    | root_before_updating_query_text.query[:].text.get_data_slice(),
    'Updated the text of the first query to be about weight',
)

In [237]:
root_before_updating_query_text.query[:].text.get_data_slice()

DataSlice(['How heavy is the Eiffel Tower', 'How high is the empire state building'], schema: STRING, present: 2/2, bag_id: $2a1a)

In [238]:
root.query[:].text.get_data_slice()

DataSlice(['How high is the Eiffel tower in Paris', 'How high is the empire state building'], schema: STRING, present: 2/2, bag_id: $5812)

### Branching to filter data

Creating a branch for filtering data is useful because then the filtering updates do not update the original data/manager.

The filtering of a persisted incremental DataSlice is an update operation. It keeps only the data for which the given mask is `kd.present`, and removes empty ancestors along the view's DataSlicePath up to the root.

In [239]:
branch_manager = manager.branch(
    tempfile.mkdtemp(), description='Branch to filter data'
)

branch_root = DataSliceManagerView(branch_manager)
branch_query = branch_root.query[:]
branch_doc = branch_query.doc[:]

branch_doc.filter(
    kd.strings.contains(branch_doc.title.get_data_slice(), 'Paris'),
    description='Filter docs to keep only those with "Paris" in the title',
)

The filtering code above loaded only the data that was needed to perform the filtering.

In [240]:
print_diagnostics(branch_manager, 'After filtering')

After filtering
Managed bags: 16   triples: 44   size on disk: 0.01 Mb
Loaded bags:  10 (62.50%)   triples: 24 (54.55%)  size on disk: 0.00 Mb (32.79%)



In [241]:
branch_root.get_data_slice(populate_including_descendants=[branch_root])

DataItem(Entity(
  query=List[
    Entity(
      doc=List[
        Entity(
          content='foofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofo'...'oofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoo',
          id=10,
          title='Attractions of Paris',
        ),
      ],
      id=1,
      locale='en-GB',
      text='How high is the Eiffel tower in Paris',
    ),
  ],
), schema: ENTITY(
  query=LIST[query(
    doc=LIST[doc(content=STRING, id=INT32, title=STRING)],
    id=INT32,
    locale=STRING,
    text=STRING,
  )],
), bag_id: $1077)

In [242]:
for index, revision_metadata in enumerate(
    branch_manager.get_revision_history(tz=pytz.timezone('Europe/Zurich'))
):
  print(
      f'{index}. {type(revision_metadata).__name__}:'
      f' {revision_metadata.description}'
  )

0. BranchMetadata: Branch to filter data
1. AttributeUpdateMetadata: Filter docs to keep only those with "Paris" in the title


## Views are aware of the full schema

Each view knows about its parent (unless it is the root), its children and the root.

In [243]:
assert query.get_grandparent() == root

In [244]:
for child in query.get_children():
  print(child.get_path_from_root())
  assert child.get_parent() == query

.query[:].doc
.query[:].id
.query[:].locale
.query[:].text


In [245]:
doc = query.doc[:]

assert doc.title.get_parent() == doc

It's easy to search for views.

In [246]:
for view in query.grep_descendants('content'):
  print(view.get_path_from_root())

.query[:].doc[:].content


In [247]:
# Find all STRING "features" of a query. Do not go into docs etc.
for view in query.find_descendants(
    lambda v: v.get_schema() == kd.STRING, max_delta_depth=1
):
  print(view.get_path_from_root())

.query[:].locale
.query[:].text


Operations on views are checked immediately against the schema. Errors are flagged immediately.

In [248]:
try:
  root[:]
except ValueError as e:
  print(e)

invalid data slice path: '[:]'


In [249]:
try:
  query.foo
except ValueError as e:
  print(e)

invalid data slice path: '.query[:].foo'


Views give you the ability to talk about an entire feature/column. Views do not support operations that involve anything less than that. Instead, such operations have to be performed on the DataSlices, i.e. after calling `view.get_data_slice()`.

In [250]:
try:
  root.query[0]
except ValueError as e:
  print(e)

only the [:] syntax is supported; got a request for [0]


In [251]:
root.query.get_data_slice(populate_including_descendants=[root.query])[0]

DataItem(Entity(
  doc=List[
    Entity(
      content='foofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofo'...'oofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoofoo',
      id=10,
      title='Attractions of Paris',
    ),
  ],
  id=1,
  locale='en-GB',
  text='How high is the Eiffel tower in Paris',
), schema: query(
  doc=LIST[doc(content=STRING, id=INT32, title=STRING)],
  id=INT32,
  locale=STRING,
  text=STRING,
), bag_id: $9c26)

When you type `view.<tab>` in a notebook, auto-complete will suggest a schema-aware list of methods.

Colab has some limitations: after long chains of operations on a view, it may not give the proper completions anymore. Saving the long chain in a variable seems to solve that. Auto-complete will suggest good completions for the variable and short chains of operations on it.

## Loading data sequentially and in parallel

In [252]:
def get_data_sequential(
    manager: PersistedIncrementalDataSliceManager,
) -> dict[str, kd.types.DataSlice]:
  root = DataSliceManagerView(manager)
  query = root.query[:]
  doc = query.doc[:]

  return dict(
      # The calls to get_data_slice() here are run in sequence. However, the
      # index structure of queries and docs is cached across the calls.
      query_text=query.text.get_data_slice(),
      doc_title=doc.title.get_data_slice(),
      doc_id=doc.id.get_data_slice(),
  )


def convert_to_pure_python(data: dict[str, kd.types.DataSlice]):
  return kd.new(**data).flatten().to_py(obj_as_dict=True)

In [253]:
manager.clear_cache()

In [254]:
%%time

data_sequential = get_data_sequential(manager)

CPU times: user 10.9 ms, sys: 203 µs, total: 11.1 ms
Wall time: 10.3 ms


In [255]:
def get_data_parallel(
    manager: PersistedIncrementalDataSliceManager,
) -> dict[str, kd.types.DataSlice]:
  root = DataSliceManagerView(manager)
  query = root.query[:]
  doc = query.doc[:]

  needed_views = {
      'query_text': query.text,
      'doc_title': doc.title,
      'doc_id': doc.id,
  }
  # Here is the crux of parallel loading: we ask for a DataSlice that is
  # populated with an arbitrary set of features. The needed bags are loaded in
  # parallel from disk and then cached.
  root.get(populate=needed_views.values())
  # Now we just pick up the loaded bags from the cache:
  return {name: view.get() for name, view in needed_views.items()}

Because our data is so small, the results are very noisy, and the parallel loading isn't always faster. But on bigger examples the difference is much more pronounced.

In [256]:
manager.clear_cache()

In [257]:
%%time

data_parallel = get_data_parallel(manager)

CPU times: user 11.7 ms, sys: 1.73 ms, total: 13.5 ms
Wall time: 11.5 ms


In [258]:
assert convert_to_pure_python(data_sequential) == convert_to_pure_python(
    data_parallel
)

## Restrictions

PersistedIncrementalDataSliceManager indexes the data by considering only its schema. Roughly speaking, it is a column-based storage abstraction.

The use of kd.OBJECT schema is banned in incremental data, because the schema
(kd.OBJECT) does not contain the information about the structure of the data that is needed for proper indexing.

The next example shows how OBJECTs can hide information about their true character: based on the schema alone, it is not possible to infer that the update does not only add the attribute `root.naughty_obj`, but also `root.foo`. Updates that use kd.OBJECT somewhere are always rejected outright.

In [259]:
try:
  root.naughty_obj = (
      kd.obj(root.get_data_slice().with_attrs(foo=123)),
      'Trying to add an OBJECT',
  )
except ValueError as e:
  print(e)

OBJECT schemas are not supported. Please use a structured schema, or remove the data, or serialize it and attach it as BYTES data instead


A somewhat unfortunate consequence of banning OBJECT is that Koda functors can only be stored in serialized form in incremental DataSlices.

Additionally, the manager enforces a complete separation between data and schema. That is needed to make its reasoning about updates, which is based only on the schema of the data, consistent with the behavior of analogous updates in standard Koda DataSlices (i.e. non-incremental DataSlices).

In particular, schemas cannot be stored inside data, i.e. kd.SCHEMA is banned. The next example shows why: not only does it add an attribute `root.my_naughty_schema` with schema `kd.SCHEMA`, but it also covertly adds an attribute `root.foo`:

In [260]:
try:
  root.my_naughty_schema = root.get_schema().with_attr('foo', kd.INT32)
except ValueError as e:
  print(e)

SCHEMA schemas are not supported. Please remove it and try again. If you truly want to include data with schema SCHEMA, then consider serializing it and attaching the resulting BYTES data instead


The separation between data and schema is complete. Not only can data not include SCHEMAs, but SCHEMAs cannot include data.

In particular, non-primitive data cannot be piggy-backed on schemas (schema metadata, to be precise). The next example shows why: not only does the update add an attribute `root.my_naughty_data`, but it also covertly adds an attribute `foo` to the first query.

In [261]:
first_query = query.get_data_slice().S[0]

innocent_looking_schema = kd.named_schema('i_am_innocent')
innocent_looking_schema = kd.with_metadata(
    innocent_looking_schema, covert_data=first_query.with_attrs(foo='bar')
)

try:
  root.my_naughty_data = (
      innocent_looking_schema.new(a=123),
      'Covert data updates are banned',
  )
except ValueError as e:
  print(e)

schema i_am_innocent(a=INT32, __schema_metadata__=Obj(covert_data=Entity(foo='bar'))) has metadata attributes that are not primitives


## Aliases and graph data

Although persisted incremental DataSlices are rooted, they do not have to be trees. Aliases and graph data are fully supported.

In [262]:
def aliasing_example():
  e = kd.new(x=kd.new(z=1))

  manager = PersistedIncrementalDataSliceManager.create_new(tempfile.mkdtemp())
  root = DataSliceManagerView(manager)

  root.e1 = e, 'Added data for e1'
  root.e2 = e, 'Made e2 an alias for e1'
  root.e3 = root.e2.get_data_slice(), 'Made e3 an alias for e2'
  root.e4 = e.stub(), 'Yet another alias'

  e1 = root.e1
  e2 = root.e2
  e3 = root.e3
  e4 = root.e4

  # Next, we add a new feature to e1. It is implicitly added to e2, e3 and e4 as
  # well because of the aliasing.
  e1.y = kd.item(2)

  for view in [e1, e2, e3, e4]:
    assert view.y.get_data_slice().to_py() == 2

  # Starting from a state where nothing is loaded, the manager still knows
  # that "y" is a descendant of e1, e2, e3, and e4.
  manager.clear_cache()
  for view in [e1, e2, e3, e4]:
    assert view.y.get_data_slice().to_py() == 2

  # The same is also true for completely new instances that read the state from
  # the persistence directory.
  new_manager = PersistedIncrementalDataSliceManager.create_from_dir(
      manager.get_persistence_directory()
  )
  new_root = DataSliceManagerView(new_manager)
  assert new_root.e4.y.get_data_slice().to_py() == 2


aliasing_example()

In [263]:
def graph_data_example():
  graph_node_schema = kd.named_schema(
      'GraphNode',
      label=kd.STRING,
      outgoing_edges=kd.list_schema(kd.named_schema('GraphNode')),
  )
  node1 = graph_node_schema.new(label='node1')
  node2 = graph_node_schema.new(label='node2', outgoing_edges=kd.list([node1]))
  node1 = node1.updated(kd.attrs(node1, outgoing_edges=kd.list([node2])))

  kd.testing.assert_equivalent(
      node1.outgoing_edges[:].outgoing_edges[:].flatten(),
      kd.slice([node1]),
  )

  manager = PersistedIncrementalDataSliceManager.create_new(tempfile.mkdtemp())
  root = DataSliceManagerView(manager)

  root.graph_nodes = (
      kd.list([node1, node2]),
      'Populated graph_nodes with node1 and node2',
  )
  kd.testing.assert_equivalent(
      root.graph_nodes[:]
      .outgoing_edges[:]
      .outgoing_edges[:]
      .label.get_data_slice()
      .flatten(),
      kd.slice(['node1', 'node2']),
  )
  kd.testing.assert_equivalent(
      root.graph_nodes[:].outgoing_edges[:].label.get_data_slice().flatten(),
      kd.slice(['node2', 'node1']),
  )

  graph_nodes = root.graph_nodes[:]
  outgoing_edges_lists = graph_nodes.outgoing_edges
  graph_nodes.outgoing_edges = (
      kd.reverse(outgoing_edges_lists.get_data_slice()),
      'Changed both outgoing edges into self-edges by swapping the lists',
  )
  kd.testing.assert_equivalent(
      root.graph_nodes[:].outgoing_edges[:].label.get_data_slice().flatten(),
      kd.slice(['node1', 'node2']),
  )


graph_data_example()